## Import library

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import yaml
import re
import matplotlib.pyplot as plt

# pd.set_option('future.no_silent_downcasting', True)

## Load data and tokens

In [2]:
with open('data/entities.json', 'r') as file:
    data = json.load(file)

with open('data/tokens.yml', 'r') as file:
    tokens = yaml.safe_load(file)
    
token_dict = {}
for col in tokens.keys():
    token = tokens[col]['start']
    token_dict[token] = col

token_dict

{'Ⓐ': 'age',
 'Ⓑ': 'birth_date',
 'Ⓒ': 'civil_status',
 'Ⓓ': 'education_level',
 'Ⓔ': 'employer',
 'Ⓕ': 'firstname',
 'Ⓗ': 'link',
 'Ⓘ': 'lob',
 'Ⓙ': 'maiden_name',
 'Ⓚ': 'nationality',
 'Ⓛ': 'observation',
 'Ⓜ': 'occupation',
 'Ⓞ': 'surname',
 'Ⓟ': 'surname_household'}

In [3]:
## Some Useful functions in my own .py

In [4]:
from src.useful_functions import *

## Preprocessing

In [5]:
df = get_preprocessing_done(data, tokens,  token_dict)
df = df.rename(columns={"surname_household": "Is_household"})
X = df.drop(columns=['Is_household'])
y = df['Is_household'].apply(
                                lambda x: 0 if pd.isna(x) else 1
                            )

100%|██████████| 1218/1218 [00:00<00:00, 3335.77it/s]
/home/onyxia/work/useful_functions.py:53: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame().from_dict(df_dict, orient='index').fillna(value=np.nan)


In [6]:
X

,age,birth_date,civil_status,education_level,employer,firstname,link,lob,maiden_name,nationality,observation,occupation,surname
0,25,NaN,Garçon,NaN,NaN,Cyrille,NaN,NaN,NaN,française,NaN,menuisier,Breton
1,30,NaN,Garçon,NaN,NaN,Auguste,NaN,NaN,NaN,Piémontaise,NaN,vitrier,NaN
2,24,NaN,Garçon,NaN,NaN,Pierre,NaN,NaN,NaN,Piémontaise,NaN,vitrier,NaN
3,48,NaN,Homme marié,NaN,NaN,Alexandre,NaN,NaN,NaN,française,NaN,prop re,NaN
4,30,NaN,NaN,NaN,NaN,Zélie,sa fe,NaN,NaN,française,NaN,prop re,Vignat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25433,NaN,1869,NaN,NaN,NaN,Marie,chef,Pailharès,NaN,idem,NaN,NaN,NaN
25434,NaN,1863,NaN,NaN,Cara,Marie,chef,St Naz en Royans,NaN,idem,NaN,ouv chaus res,NaN
25435,NaN,1886,NaN,NaN,Baretto,Nello,chef,Castel,NaN,italienne,NaN,manoeuvre,NaN
25436,NaN,1887,NaN,NaN,NaN,Annunziata,épouse,idem,NaN,idem,NaN,NaN,Berni-Laureti


In [7]:
y

0        0
1        1
2        1
3        1
4        0
        ..
25433    1
25434    1
25435    1
25436    0
25437    0
Name: Is_household, Length: 25074, dtype: int64

## Gradient Boosting

In [8]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import log_loss

#### Ordinal Encoding

In [9]:
enc = OrdinalEncoder()
X_encoded = enc.fit_transform(X)

In [10]:
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(X_encoded, y, test_size=0.20, stratify=y)

### Gradient Boosting

In [11]:
param_grid = {
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "max_leaf_nodes": [15, 31, 100],
    "max_depth": [None, 5, 10, 20],
    "min_samples_leaf": [10, 20, 50],
    "l2_regularization": [0.0, 0.1, 1.0]
}

model = HistGradientBoostingClassifier(max_iter=10_000, early_stopping=True, class_weight='balanced', validation_fraction=0.2)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=10)

grid_search.fit(X_train_enc, y_train_enc)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV 1/3; 7/324] START l2_regularization=0.0, learning_rate=0.1, max_depth=None, max_leaf_nodes=100, min_samples_leaf=10
[CV 1/3; 7/324] END l2_regularization=0.0, learning_rate=0.1, max_depth=None, max_leaf_nodes=100, min_samples_leaf=10;, score=0.996 total time=   1.0s
[CV 1/3; 32/324] START l2_regularization=0.0, learning_rate=0.1, max_depth=20, max_leaf_nodes=31, min_samples_leaf=20
[CV 1/3; 32/324] END l2_regularization=0.0, learning_rate=0.1, max_depth=20, max_leaf_nodes=31, min_samples_leaf=20;, score=0.996 total time=   0.4s
[CV 1/3; 42/324] START l2_regularization=0.0, learning_rate=0.01, max_depth=None, max_leaf_nodes=31, min_samples_leaf=50
[CV 1/3; 42/324] END l2_regularization=0.0, learning_rate=0.01, max_depth=None, max_leaf_nodes=31, min_samples_leaf=50;, score=0.995 total time=   3.5s
[CV 3/3; 52/324] START l2_regularization=0.0, learning_rate=0.01, max_depth=5, max_leaf_nodes=100, min_samples_leaf=10
[CV 3/3

GridSearchCV(cv=3,
             estimator=HistGradientBoostingClassifier(class_weight='balanced',
                                                      early_stopping=True,
                                                      max_iter=10000,
                                                      validation_fraction=0.2),
             n_jobs=-1,
             param_grid={'l2_regularization': [0.0, 0.1, 1.0],
                         'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [None, 5, 10, 20],
                         'max_leaf_nodes': [15, 31, 100],
                         'min_samples_leaf': [10, 20, 50]},
             scoring='accuracy', verbose=10)

In [12]:
# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

Best parameters: {'l2_regularization': 0.0, 'learning_rate': 0.1, 'max_depth': 10, 'max_leaf_nodes': 100, 'min_samples_leaf': 10}


In [13]:
# Get the best model from the grid search
clf = grid_search.best_estimator_
clf.fit(X_train_enc, y_train_enc)

HistGradientBoostingClassifier(class_weight='balanced', early_stopping=True,
                               max_depth=10, max_iter=10000, max_leaf_nodes=100,
                               min_samples_leaf=10, validation_fraction=0.2)

In [14]:
print(f"Train accuracy: {clf.score(X_train_enc, y_train_enc):.3f}")
print(f"Test accuracy: {clf.score(X_test_enc, y_test_enc):.3f}")

Train accuracy: 0.999
Test accuracy: 0.996


In [15]:
print(f'Train CE loss = {log_loss(y_train_enc, clf.predict_proba(X_train_enc)):.4f}')
print(f'Test CE loss = {log_loss(y_test_enc, clf.predict_proba(X_test_enc)):.4f}')

Train CE loss = 0.0038
Test CE loss = 0.0179


## Encode through pre-trained model

In [16]:
from transformers import BartTokenizer, BartModel

ModuleNotFoundError: No module named 'transformers'

In [ ]:
X_str = np.array([' '.join([str(x) for x in X.iloc[i].dropna(inplace=False).values]) for i in range(len(X))], dtype=str)

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartModel.from_pretrained('facebook/bart-base')

In [ ]:
inputs

In [ ]:
inputs = tokenizer(X_str[10], return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [ ]:
last_hidden_states.cpu().detach().numpy().squeeze().shape